# Intro to Biologging data 
- make sure to install https://github.com/IAOCEA/pangeo-fish before running this example

In [ ]:
from pangeo_fish.io import open_tag
from pangeo_fish.tags import to_time_slice

In [ ]:
# tag_name corresponds to the name of the biologging tag name (DST identification number),
# which is also a path for storing all the information for the specific fish tagged with tag_name.
tag_name = "A19124"

# tag_root specifies the root URL for tag data used for this computation.
tag_root = "https://data-taos.ifremer.fr/data_tmp/cleaned/tag/"

target_root = "./{tag_name}"

## Reading tag data
- If the cell below fails you may not have read access to the corresponding tag data; create a [GitHub issue](https://github.com/destination-earth/DestinE_ESA_GFTS).

In [ ]:
# Open and retrieve the tag data required for the analysis
tag = open_tag(tag_root, tag_name)
tag

In [ ]:
import hvplot.xarray  # noqa
from pangeo_fish.io import save_html_hvplot

# Drop tag data outside the tagged events interval
time_slice = to_time_slice(tag["tagging_events/time"])
tag_log = tag["dst"].ds.sel(time=time_slice)

plot = (
    (-tag["dst"].pressure).hvplot(width=1000, height=500, color="blue")
    * (-tag_log).hvplot.scatter(
        x="time", y="pressure", color="red", size=5, width=1000, height=500
    )
    * (
        (tag["dst"].temperature).hvplot(width=1000, height=500, color="blue")
        * (tag_log).hvplot.scatter(
            x="time", y="temperature", color="red", size=5, width=1000, height=500
        )
    )
)
filepath = f"{target_root}/tags.html"

save_html_hvplot(plot, filepath)

plot

## Write results into the IFREMER bucket
- If it fails and you believe you should have write access to the IFREMER S3 bucket, please write a [GitHub issue](https://github.com/destination-earth/DestinE_ESA_GFTS).

In [ ]:
scratch_root = "s3://destine-gfts-data-lake/demo"

# Define target root directories for storing analysis results.
target_root = f"{scratch_root}/{tag_name}"

# storage_options specifies options for the filesystem storing output files.
storage_options = {
    "anon": False,
    "profile": "gfts",
    "client_kwargs": {
        "endpoint_url": "https://s3.gra.perf.cloud.ovh.net",
        "region_name": "gra",
    },
}

In [ ]:
filepath = f"{target_root}/tags.html"

In [ ]:
save_html_hvplot(plot, filepath, storage_options)

## Check the remote file

In [ ]:
import s3fs

In [ ]:
s3 = s3fs.S3FileSystem(**storage_options)

In [ ]:
s3.ls(filepath)